In [15]:
#create container
import boto3
import time
ec2 = boto3.resource('ec2')
ec2_client = boto3.client('ec2')
autoscaling = boto3.client('autoscaling')
ecs = boto3.client('ecs')
r53 = boto3.client('route53')
launchtemplate = 'EC2ContainerService-sysvis-EcsInstanceLc-1H1654961ZY8F'
autoscalinggroup = 'EC2ContainerService-sysvis-EcsInstanceAsg-196WYOOZAFI87'
vpc = 'vpc-0d7c412110e5f49f2'
subnet = 'subnet-0596881f608fd4f70'
securitygroup = 'sg-05c4123649a439051'
hosted_zone_id = "Z09241041ZJLPUFRNRNGX"
instance_id = ec2.create_instances(MinCount=1, MaxCount=1, LaunchTemplate={'LaunchTemplateName':launchtemplate}, SecurityGroupIds=[securitygroup], SubnetId=subnet)
instance_id = instance_id[0].instance_id
print('instance id', instance_id)
ready = False
time.sleep(2)
while not ready:
    if ec2_client.describe_instances(InstanceIds=[instance_id])['Reservations'][0]['Instances'][0]['State']['Name'] =='running':
        ready = True
time.sleep(2)
autoscaling.attach_instances(AutoScalingGroupName=autoscalinggroup, InstanceIds=[instance_id])
ready = False
print('created')
while not ready:
    response = autoscaling.describe_auto_scaling_groups(AutoScalingGroupNames=[autoscalinggroup])
    realresponse = response['AutoScalingGroups'][0]
    instances = realresponse['Instances']
    for i in instances:
        if i['InstanceId'] == instance_id:
            ready = True
print('attached')
ready = False
while not ready:
    containers = ecs.list_container_instances(cluster='sysvis')['containerInstanceArns']
    for i in containers:
        container = i
        findinstanceid = ecs.describe_container_instances(cluster='sysvis',containerInstances=[container])['containerInstances'][0]['ec2InstanceId']
        if findinstanceid == instance_id:
            ready = True
task = ecs.start_task(cluster='sysvis', taskDefinition='sysvis', containerInstances=[container])
time.sleep(2)
task_arn = task['tasks'][0]['taskArn']
ready = False
print('container')
while not ready:
    status = ecs.describe_tasks(cluster = 'sysvis', tasks = [task_arn])['tasks'][0]['lastStatus']
    if status == 'RUNNING':
        ready = True
print('finished')
print('Ip Address', ec2_client.describe_instances(InstanceIds=[instance_id])['Reservations'][0]['Instances'][0]['PublicIpAddress'])
ip_address = ec2_client.describe_instances(InstanceIds=[instance_id])['Reservations'][0]['Instances'][0]['PublicIpAddress']
print('Public Dns Name', ec2_client.describe_instances(InstanceIds=[instance_id])['Reservations'][0]['Instances'][0]['PublicDnsName'])
url = instance_id[2:] + ".synesthesiavisualizer.com"
record = r53.change_resource_record_sets(HostedZoneId=hosted_zone_id, ChangeBatch={'Changes': [{'Action': 'CREATE','ResourceRecordSet': {'Name': url,'Type': 'A', 'TTL':300, 'ResourceRecords': [{'Value': ip_address},],}},]})
print('zone')


instance id i-0e0be7ba52b332b9c


created
attached


{'tasks': [{'attachments': [], 'availabilityZone': 'us-east-1a', 'clusterArn': 'arn:aws:ecs:us-east-1:351629464041:cluster/sysvis', 'containerInstanceArn': 'arn:aws:ecs:us-east-1:351629464041:container-instance/sysvis/f85d9cdf3bf34d95aa88c39b27be8f0d', 'containers': [{'containerArn': 'arn:aws:ecs:us-east-1:351629464041:container/0bdf3b42-0a6b-469d-bce9-48c071fbca2a', 'taskArn': 'arn:aws:ecs:us-east-1:351629464041:task/sysvis/6dd42247440c4c5eaaf71ee06e7e492a', 'name': 'sysvis', 'image': '351629464041.dkr.ecr.us-east-1.amazonaws.com/sysvis', 'lastStatus': 'PENDING', 'networkInterfaces': [], 'cpu': '0', 'memory': '15258'}], 'cpu': '0', 'createdAt': datetime.datetime(2020, 11, 28, 19, 38, 22, 496000, tzinfo=tzlocal()), 'desiredStatus': 'RUNNING', 'group': 'family:sysvis', 'lastStatus': 'PENDING', 'launchType': 'EC2', 'memory': '15258', 'overrides': {'containerOverrides': [{'name': 'sysvis'}], 'inferenceAcceleratorOverrides': []}, 'tags': [], 'taskArn': 'arn:aws:ecs:us-east-1:351629464041:t

container


finished
Ip Address 54.174.253.49


Public Dns Name ec2-54-174-253-49.compute-1.amazonaws.com


zone


In [28]:
#delete container
import boto3
autoscaling = boto3.client('autoscaling')
ec2_client = boto3.client('ec2')
instance_id = 'i-076613f4eb39cf069'
autoscalinggroup = 'EC2ContainerService-sysvis-EcsInstanceAsg-196WYOOZAFI87'
autoscaling = boto3.client('autoscaling')
try:
    autoscaling = autoscaling.detach_instances(InstanceIds=[instance_id],AutoScalingGroupName=autoscalinggroup, ShouldDecrementDesiredCapacity=True)
except:
    print("not attached")
url = instance_id[2:] + ".synesthesiavisualizer.com"
try:
    record = r53.change_resource_record_sets(HostedZoneId=hosted_zone_id, ChangeBatch={'Changes': [{'Action': 'DELETE','ResourceRecordSet': {'Name': url,'Type': 'A', 'TTL':300, 'ResourceRecords': [{'Value': ip_address},],}},]})
except:
    print("no record")
print(response)
response = ec2_client.terminate_instances(InstanceIds=[instance_id]) 

no record
{'TerminatingInstances': [{'CurrentState': {'Code': 32, 'Name': 'shutting-down'}, 'InstanceId': 'i-0a0f762d278626f83', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': 'cbe29e0f-98ed-4d35-bb17-a96443bb0bc3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'cbe29e0f-98ed-4d35-bb17-a96443bb0bc3', 'content-type': 'text/xml;charset=UTF-8', 'transfer-encoding': 'chunked', 'vary': 'accept-encoding', 'date': 'Sun, 29 Nov 2020 19:20:06 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}
